# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import holoviews as hv
import geoviews as gv
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

/Users/ryak/anaconda3/envs/dev/lib/python3.10/site-packages/geoviews/operation/__init__.py:14: HoloviewsDeprecationWarning: 'ResamplingOperation' is deprecated and will be removed in version 1.17, use 'ResampleOperation2D' instead.
  from holoviews.operation.datashader import (


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,286.46,92,75,6.78,GB,2023-07-15 21:00:00
1,1,longyearbyen,78.2186,15.6401,285.06,62,0,0.37,SJ,2023-07-15 21:00:00
2,2,blackmans bay,-43.0167,147.3167,284.82,69,99,6.54,AU,2023-07-15 21:00:00
3,3,ulaangom,49.9811,92.0667,293.54,48,48,1.17,MN,2023-07-15 21:00:00
4,4,wailua homesteads,22.0669,-159.3780,300.04,70,75,4.33,US,2023-07-15 21:00:00


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
    
# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
#Convert Kelvin to Fahrenheit for max temp:
city_data_df['Max Temp_F'] = (city_data_df['Max Temp']-273.15)*9/5+32

# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df['Max Temp_F'] >= 65) & (city_data_df['Max Temp_F'] <= 85) & (city_data_df['Wind Speed'] <= 10) & (city_data_df['Cloudiness'] <= 20) & (city_data_df['Humidity'] <= 40)]

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Display sample data
ideal_cities_df.head()

/var/folders/pr/r9s7fv8d3hl7zl078q3rzs_00000gn/T/ipykernel_6626/2964795537.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp_F
41,41,copiapo,-27.3667,-70.3333,296.06,30,0,2.25,CL,2023-07-15 21:00:00,73.238
72,72,monte patria,-30.6919,-70.9467,295.49,18,0,1.92,CL,2023-07-15 21:00:00,72.212
75,75,buka,40.8108,69.1986,294.97,22,0,2.09,UZ,2023-07-15 21:00:00,71.276
86,86,glendive,47.1053,-104.7125,300.25,40,0,8.13,US,2023-07-15 21:00:00,80.780
119,119,harsin,34.2721,47.5861,296.60,18,0,0.87,IR,2023-07-15 21:00:00,74.210


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,stanley,GB,54.8680,-1.6985,92,
1,longyearbyen,SJ,78.2186,15.6401,62,
2,blackmans bay,AU,-43.0167,147.3167,69,
3,ulaangom,MN,49.9811,92.0667,48,
4,wailua homesteads,US,22.0669,-159.3780,70,
...,...,...,...,...,...,...
561,shunyi,CN,40.1250,116.6453,50,
562,watampone,ID,-4.5386,120.3279,89,
563,kiffa,MR,16.6166,-11.4045,35,
564,dwarka,IN,22.2394,68.9678,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    #print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
stanley - nearest hotel: Hotel 52
longyearbyen - nearest hotel: The Vault
blackmans bay - nearest hotel: Villa Howden
ulaangom - nearest hotel: АНДУУД
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
port-aux-francais - nearest hotel: Keravel
amagasaki - nearest hotel: 東横イン
daru - nearest hotel: Kuki
ormara - nearest hotel: No hotel found
anadyr - nearest hotel: Гостевой дом
grytviken - nearest hotel: No hotel found
ampanihy - nearest hotel: Hotel Restaurant Maison-Blance
hinton - nearest hotel: Timberland Hotel
geraldton - nearest hotel: Ocean Centre Hotel
egilsstadir - nearest hotel: Valaskjálf
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
kaiama - nearest hotel: No hotel found
sasolburg - nearest hotel: Be My Guest Hotel
codrington - nearest hotel: No hotel found
lagos - nearest hotel: No hotel found
chonchi - nearest hotel: Hotel Huildín
bredasdorp - nearest hotel: Victoria Hotel
inhambane - nearest hotel: Hotel de Inhambane

tsiombe - nearest hotel: No hotel found
la ronge - nearest hotel: La Ronge Hotel and Suites
farrokh shahr - nearest hotel: هتل و رستوران خلیج فارس
santiago de cao - nearest hotel: Hostal El Encanto
corinto - nearest hotel: Apollon
san justo - nearest hotel: Hotel España
axim - nearest hotel: Axim Beach Hotel
puqi - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
maneromango - nearest hotel: No hotel found
ceeldheer - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
port augusta - nearest hotel: Hotel Commonwealth
fale old settlement - nearest hotel: No hotel found
'ain el hadjar - nearest hotel: Hôtel Forsane
kapuskasing - nearest hotel: Travelodge
elesbao veloso - nearest hotel: No hotel found
cartagena del chaira - nearest hotel: Los Corceles
kabalo - nearest hotel: No hotel found
ascension - nearest hotel: Cajun Country Inn
port-vila - nearest hotel: Holiday Inn Resort Vanuatu
kununurra - nearest hotel: Kununurra Country Club Resort
so

naze - nearest hotel: ビジネス旅館畠山
kavaratti - nearest hotel: No hotel found
north branch - nearest hotel: AmericInn
fort bragg - nearest hotel: Airborne Inn Lodging
novhorod-sivers'kyy - nearest hotel: ПаСвІрДа
selfoss - nearest hotel: Miðgarður
thabazimbi - nearest hotel: Maroela Guest Lodge
tekkali - nearest hotel: No hotel found
pandan - nearest hotel: No hotel found
batang - nearest hotel: Palm Beach Hotel
tual - nearest hotel: Suita
bardai - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
cabo san lucas - nearest hotel: Comfort Rooms
longboat key - nearest hotel: Zota Beach Resort
wainwright - nearest hotel: Park Hotel
sault ste. marie - nearest hotel: Holiday Inn
pugachev - nearest hotel: Гостевой дом
ixtapa - nearest hotel: Ma. Cristina
amriswil - nearest hotel: Gasthaus zum Ochsen
nabire - nearest hotel: Hotel Anggrek
marsabit - nearest hotel: Chicho Hotel
chizhou - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
as samawah - 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,stanley,GB,54.8680,-1.6985,92,Hotel 52
1,longyearbyen,SJ,78.2186,15.6401,62,The Vault
2,blackmans bay,AU,-43.0167,147.3167,69,Villa Howden
3,ulaangom,MN,49.9811,92.0667,48,АНДУУД
4,wailua homesteads,US,22.0669,-159.3780,70,Hilton Garden Inn Kauai Wailua Bay
...,...,...,...,...,...,...
561,shunyi,CN,40.1250,116.6453,50,会议楼
562,watampone,ID,-4.5386,120.3279,89,Hotel Rio Rita
563,kiffa,MR,16.6166,-11.4045,35,فندق كيفة
564,dwarka,IN,22.2394,68.9678,79,The Dwarika Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

#Convert the 'Hotel Name' to a string
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].astype(str)

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols =["Hotel Name","Country"]
)
    
# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)